In [94]:
# others
import pandas as pd
import numpy as np
import argparse
import datetime
from copy import deepcopy # Add Deepcopy for args
import pickle 
import seaborn as sns 
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error

# pytorch
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim

# tensorflow
import tensorflow as tf

print(torch.__version__)
%matplotlib inline
%pylab inline
mpl.rcParams['figure.figsize'] = (8, 6)
mpl.rcParams['axes.grid'] = False

# read file
file_path = './data/merged_data.h5'
df = pd.read_hdf(file_path)


1.6.0+cpu
Populating the interactive namespace from numpy and matplotlib


C:\Anaconda3\envs\venv\lib\site-packages\IPython\core\magics\pylab.py:160: UserWarning: pylab import has clobbered these variables: ['seed', 'partition']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


In [95]:
def df2d_to_array3d(df_2d):
    
    # 입력 받은 2차원 데이터 프레임을 3차원 numpy array로 변경하는 함수
    feature_size = df_2d.iloc[:,2:].shape[1]
    time_size = len(df_2d.time.value_counts())
    sample_size = len(df_2d.sample_id.value_counts())
    array_3d = df_2d.iloc[:,2:].values.reshape([sample_size, time_size, feature_size])
    
    return array_3d

In [96]:
def train_val_test_spliter(arr):
    
    n = len(arr)
    num_features = arr.shape[2] - 1
    
    train_arr = arr[0:int(n*0.7), :, 1:5]
    val_arr = arr[int(n*0.7):int(n*0.9), :, 1:5]
    test_arr = arr[int(n*0.9):, : , 1:5]
    
    n2 = len(train_arr) + len(val_arr) + len(test_arr)
    
    print(
    f'''
    ======================================================
    Origin length is {n}, then total split length is {n2}
    ======================================================
    train length is {train_arr.shape},
    val length is {val_arr.shape},
    test length is {test_arr.shape},
    num_features is ({num_features})
    '''
    )
    
    return train_arr, val_arr, test_arr, num_features

In [97]:
raw_array = df2d_to_array3d(df)
# print(f'raw array shape is {raw_array.shape}')
train_arr, val_arr, test_arr, num_features = train_val_test_spliter(raw_array)


    Origin length is 1208, then total split length is 1208
    train length is (845, 1500, 4),
    val length is (242, 1500, 4),
    test length is (121, 1500, 4),
    num_features is (9)
    


In [98]:
df = df.drop(columns = ['sample_id', 'time', 'coin_index'])
column_indices = {name : i for i, name in enumerate(df.columns)}

In [146]:
class WindowGenerator():
    ''' Dataset Generate'''
    def __init__(self, input_width, label_width, stride, data_arr, column_indices = column_indices,
                 shfit = None, label_columns=None):
    
        # Store the raw data
        self.data_arr = data_arr
        # Work out the label column indices.
        self.label_columns = label_columns
        if label_columns is not None:
            self.label_columns_indices = {name: i for i, name in enumerate(label_columns)}
        self.column_indices = column_indices
                
        # Work out the window parameters.
        self.input_width = input_width
        self.label_width = label_width
        self.shift = 1
        if shfit is not None:
            self.shift = shfit
        self.stride = stride
        
        self.label_start = self.input_width + self.shift
        self.total_window_size = self.label_start + self.label_width
        
        # input, label indices
        self.input_slice = slice(0, self.input_width)
        self.input_indices = np.arange(self.total_window_size)[self.input_slice]
        
        self.labels_slice = slice(self.label_start, None)
        self.label_indices = np.arange(self.total_window_size)[self.labels_slice]
        
        self.X_arr, self.y_arr = self.split_windows()
        
    def __repr__(self):
        return '\n'.join([
            f'Total window size: {self.total_window_size}',
            f'Input indices: {self.input_indices}',
            f'Label indices: {self.label_indices}',
            f'Label column name(s): {self.label_columns}'
        ])

    def split_windows(self):

        X, y = list(), list()
        sample_length = int(self.data_arr.shape[0])
        split_length = int((self.data_arr.shape[1] - self.total_window_size)/self.stride) + 1
        
        for temp_id in range(sample_length):
            for i in range(split_length):
                
                X.append(self.data_arr[temp_id, (i*self.stride) : (i*self.stride)+self.input_width])
                y.append(self.data_arr[temp_id, (i*self.stride)+self.label_start : (i*self.stride)+self.total_window_size])
        
        return np.array(X), np.array(y)

    def __len__(self):
        return len(self.X_arr)

    def __getitem__(self, idx):
        
        X = self.X_arr[idx, :, :]
        y = self.y_arr[idx, :, :]
        
        return X, y 

In [147]:
# ====== initialization
parser = argparse.ArgumentParser()
args = parser.parse_args("")
args.device = 'cuda' if torch.cuda.is_available() else 'cpu'
print("device is",args.device)

seed = 777
np.random.seed(seed)
torch.manual_seed(seed)


# ====== Model Capacity options ===== #
args.input_dim = 4
args.hidden_dim = 100
args.output_dim = 1
args.n_layers = 2
args.batch_size = 128
args.dropout = 0.2
args.use_bn = True

# ====== Dataset Generating options ====== #
args.input_width = 120
args.label_width = 120
args.stride = 10

# ====== Model training options ===== #
args.num_epoch = 10
args.learning_rate = 0.01
args.L2_rate = 0.0001

trainset = WindowGenerator(input_width=args.input_width, label_width=args.label_width, stride=args.stride, data_arr = train_arr)
valset = WindowGenerator(input_width=args.input_width, label_width=args.label_width, stride=args.stride, data_arr = val_arr)
testset = WindowGenerator(input_width=args.input_width, label_width=args.label_width, stride=args.stride, data_arr = test_arr)
partition = {'train': trainset, 'val':valset, 'test':testset}


device is cpu


In [152]:
trainloader = DataLoader(trainset, batch_size = 32, shuffle = True, drop_last = True)
for i, (X,y) in enumerate(trainloader):
    print(f'Inputs shape (batch, time, features): {X.shape}')
    print( type(tf.convert_to_tensor(X)))
    print(f'Labels shape (batch, time, features): {y.shape}')
    break

Inputs shape (batch, time, features): torch.Size([32, 120, 4])
<class 'tensorflow.python.framework.ops.EagerTensor'>
Labels shape (batch, time, features): torch.Size([32, 120, 4])


In [168]:
ds = tf.keras.preprocessing.timeseries_dataset_from_array(
      data=trainset,
      targets=None,
      sequence_length=1,
      sequence_stride=1,
      shuffle=True,
      batch_size=32,)

In [172]:
for i, (X,y) in enumerate(ds):
    print(X.shape)
    break

InvalidArgumentError: indices[0] = 39370 is not in [0, 2)
	 [[{{node GatherV2}}]]

In [164]:
OUT_STEPS = 120
num_features = 1

DNN_model = tf.keras.Sequential([
    # Take the last time-step.
    # Shape [batch, time, features] => [batch, 1, features]
    tf.keras.layers.Lambda(lambda x: x[:, -1:, :]),
    # Shape => [batch, 1, out_steps*features]
    tf.keras.layers.Dense(OUT_STEPS*num_features,
                          kernel_initializer=tf.initializers.zeros),
    # Shape => [batch, out_steps, features]
    tf.keras.layers.Reshape([OUT_STEPS, num_features])
])


early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=2, mode='min')
DNN_model.compile(loss=tf.losses.MeanSquaredError(),
                  optimizer=tf.optimizers.Adam(),
                  metrics=[tf.metrics.MeanAbsoluteError()])

In [ ]:
model.fit(window.train, epochs=MAX_EPOCHS,
                      validation_data=window.val,
                      callbacks=[early_stopping])

In [156]:
  history = 
  return history

In [ ]:

IPython.display.clear_output()
multi_val_performance['Linear'] = multi_linear_model.evaluate(multi_window.val)
multi_performance['Linear'] = multi_linear_model.evaluate(multi_window.test, verbose=0)
multi_window.plot(multi_linear_model)

In [82]:
class LSTM(nn.Module):
    
    def __init__(self, input_dim, hidden_dim, output_dim, num_layers, dropout, use_bn):
        super(LSTM, self).__init__()
        self.input_dim = input_dim 
        self.hidden_dim = hidden_dim
        self.output_dim = output_dim
        self.num_layers = num_layers

        self.dropout = dropout
        self.use_bn = use_bn 
        self.lstm = nn.LSTM(self.input_dim, self.hidden_dim, self.num_layers)

        self.regressor = self.make_regressor()
        
    def init_hidden(self, batch_size):
        return (torch.zeros(self.num_layers, batch_size, self.hidden_dim),
                torch.zeros(self.num_layers, batch_size, self.hidden_dim))
    
    def make_regressor(self):
        layers = []
        if self.use_bn:
            layers.append(nn.BatchNorm1d(self.hidden_dim))
        layers.append(nn.Dropout(self.dropout))
        
        layers.append(nn.Linear(self.hidden_dim, self.hidden_dim // 2))
        layers.append(nn.ReLU())
        layers.append(nn.Linear(self.hidden_dim // 2, self.output_dim))
        regressor = nn.Sequential(*layers)
        return regressor
    
    def forward(self, X):
        lstm_out, self.hidden = self.lstm(X)
        y_pred = self.regressor(lstm_out[-1].view(X.shape[1], -1))
        return y_pred

In [87]:
def train(model, partition, optimizer, loss_fn, args):
    ''' model training '''
   
    # data load
    trainloader = DataLoader(partition['train'],
                             batch_size = args.batch_size,
                             shuffle = True, drop_last = True)
    
    # model's mode setting
    model.train()
    model.zero_grad()
    
    train_loss = 0.0
    
    for i, (X, y) in enumerate(trainloader):
    
        X = X.transpose(0, 1).float().to(args.device)
        y_true = y[:, :, 0].float().to(args.device)
        
#         print(f'X shape is {X.shape}, y shape is {y_true.shape}')
        
        # zero the gradient
        optimizer.zero_grad()
        model.hidden = model.init_hidden(X.shape[1])

        y_pred = model(X)
#         print(f'y_pred shape is {y_pred.shape}')
        
        loss = loss_fn(y_true.view(-1), y_pred.view(-1))
        loss.backward()
        optimizer.step()
        
        # get the batch loss
        train_loss += loss.item()
        
    train_loss = train_loss / len(trainloader)
    train_loss = train_loss*10E5
    return model, train_loss



In [88]:
def validate(model, partition, loss_fn, args):
    ''' model validate '''
    
    # data load
    valloader = DataLoader(partition['val'], 
                           batch_size = args.batch_size, 
                           shuffle = False, drop_last = True)
    
    # model's mode setting
    model.eval()
    val_loss = 0.0
    
    # evaluate
    with torch.no_grad():
        for i, data in enumerate(valloader):
            
            X = X.transpose(0, 1).float().to(args.device)
            y_true = y[:, :, 0].float().to(args.device)
            
            model.hidden = model.init_hidden(X.shape[1])
            # en-decoder outputs tensor 
            y_pred = model(X)
            # compute the loss 
            loss = loss_fn(y_true.view(-1), y_pred.view(-1))

            # get the batch loss
            val_loss += loss.item()
            
    val_loss = val_loss / len(valloader)
    val_loss = val_loss*10E5
    return val_loss

In [89]:
def experiment(partition, args):

    # LSTM init is (input_dim, hidden_dim, output_dim, num_layers, dropout, use_bn):
    model = LSTM(args.input_dim, args.hidden_dim, args.label_width, args.n_layers, args.dropout, args.use_bn)
    model.to(args.device)
    
    loss_fn = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=args.learning_rate, weight_decay=args.L2_rate)
    
    # epoch-wise loss
    train_losses = []
    val_losses = []

    for epoch in range(args.num_epoch):
        
        start_time = time.time()
        model, train_loss = train(model, partition, optimizer, loss_fn, args)
        val_loss = validate(model, partition, loss_fn, args)
        end_time = time.time()
        
        # add epoch loss
        train_losses.append(train_loss)
        val_losses.append(val_loss)
        
        print('Epoch {},Loss(train/val) {:.3f}/{:.3f}. Took {:.2f} sec'.format(epoch+1, train_loss, val_loss, end_time-start_time))
    
    # test part
    # test_mae, item_loss_list = test(model, partition, scaler, args)
    
    # ======= Add Result to Dictionary ======= #
    result = {}
    
    result['train_losses'] = train_losses #epoch 수에 의존
    result['val_losses'] = val_losses 
    
    #result['test_mae'] = test_mae.round(3).item()
    
    # result['r2'] = np.array(score_list).mean().round(3)
    # item_loss = np.array(item_loss_list).mean(axis=0).mean(axis=0).astype(int)
    # item_loss = list([int(x) for x in item_loss])
    # result['item_loss'] = item_loss
     
    return vars(args), result, model

In [90]:

import IPython
import IPython.display

IPython.display.clear_output()
setting, result, model = experiment(partition, deepcopy(args))

KeyboardInterrupt: 

In [ ]:
plt.plot(result['train_losses'], label = "train loss")
plt.plot(result['val_losses'], label = "val loss")
plt.legend()
plt.show()


In [ ]:
testloader = DataLoader(partition['test'], batch_size = 1, shuffle = True, drop_last = True)

for i, X in enumerate(testloader):
    print(X[:, :args.x_frames, :].transpose(0,1).shape)
    print(X[:, args.x_frames:, 0].shape)
    
    break

model.eval()

# evaluate
with torch.no_grad():
    for i, data in enumerate(testloader):
        
        X = data[:, :args.x_frames, :].transpose(0, 1).float().to(args.device)
        y_true = data[:,args.x_frames:, 0].float().to(args.device)
        
        model.hidden = model.init_hidden(X.shape[1])

        # en-decoder outputs tensor 
        y_pred = model(X)
        
        # y values to cpu
        y_true = y_true.cpu().detach().numpy().reshape(-1)
        y_pred = y_pred.cpu().detach().numpy().reshape(-1)

        print(y_true.shape, y_pred.shape)

        plt.plot(np.arange(args.x_frames, args.x_frames+args.y_frames), y_true, label = 'True series')
        plt.plot(np.arange(args.x_frames, args.x_frames+args.y_frames), y_pred, '-',label = 'Prediction series')
        plt.legend()
        plt.show()

        
        # get the batch loss
        MAE = mean_absolute_error(y_true, y_pred)
        print(f'MAE is {MAE}')
        

In [ ]:
testloader2 = DataLoader(partition['train'], batch_size = 1, shuffle = True, drop_last = True)
model.eval()
# evaluate

with torch.no_grad():
    for i, data in enumerate(testloader2):
        print(data.shape)

        X = data[:, :args.x_frames, :].transpose(0, 1).float().to(args.device)
        y_true = data[:, args.x_frames:, 0].float().to(args.device)
        model.hidden = model.init_hidden(X.shape[1])

        # en-decoder outputs tensor 
        y_pred = model(X)
        
        # y values to cpu
        y_true = y_true.cpu().detach().numpy().reshape(-1)
        y_pred = y_pred.cpu().detach().numpy().reshape(-1)

        print(y_true.shape, y_pred.shape)

        plt.plot(np.arange(args.x_frames, args.x_frames+args.y_frames), y_true, label = 'True series')
        plt.plot(np.arange(args.x_frames, args.x_frames+args.y_frames), y_pred, '-', label = 'Prediction series')
        plt.legend()
        plt.show()

        # get the batch loss
        MAE = mean_absolute_error(y_true, y_pred)
        print(f'MAE is {MAE}')
        break
